In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
#Setting the table
pokeTable = pd.read_csv('../input/Pokemon.csv')

#Removing extra name before Mega
pokeTable = pokeTable.set_index('Name')

pokeTable.index = pokeTable.index.str.replace(".*(?=Mega)", "")

pokeTable.head(10)

In [ ]:
#Removing number (#)
pokeTable = pokeTable.drop(['#'], axis = 1)

pokeTable.head(10)

**Question 1:** I have a competitive team with the following Pokémon:
1. Azumarill *(Water/Fairy)*
2. Torterra *(Grass/Ground)*
3. Snorlax *(Normal)*
4. Emboar *(Fire/Fighting)*
5. Magnezone *(Steel/Electric)*

But I still feel as if my team is lacking. Who are the top 5 Pokémon I should consider in terms of the stats i'm lacking and the types i'm weak to?

Before proceeding further, we would first need to know the **effectiveness** of each type of Pokémon.

![](https://img.pokemondb.net/images/typechart.png)

*Data gathered from pokemondb.net*

With this information, the second half of the question can be answered, which is to identify the types this team is weak to.

Our first Pokémon on the list is **Azumarill**, a combination of *Water* and *Fairy*.  Water is *weak* to **Grass** and **Electric** while Fairy is* weak* to **Steel** and **Poison**.

Second Pokémon is **Torterra**, a combination of Grass and Ground. Grass is *weak* to Fire, Flying, Ice, Poison, and Bug while *Ground* is *weak* to **Water**, **Grass**, and **Ice**. Since **Torterra** is a combination of both *Grass* and *Ground*, it is only *weak* to **Flying**, **Poison**, **Bug**, and **Ice** since Fire is not very effective to Ground and Water is not very effective to Grass while Ice is super effective.

Third Pokémon is **Snorlax**, a *Normal* type Pokemon. Normal is *weak* to **Fighting** alone and Normal attacks has no effect on *Ghost* Pokemon.

Fourth Pokémon is **Emboar**, a combination of Fire and Fighting. Fire is *weak* to **Water**, **Ground**, and **Rock** while Fighting is weak to **Flying**, **Psychic**, and **Fairy**. However, **Rock** will not be super effective since Fighting is super effective to Rock. It will only deal normal amount of damage.

Last Pokémon is Magnezone, a combination of Electric and Steel. Electric is *weak* to **Ground** and Steel is weak to **Fire**, **Fighting**, and **Ground**. However, Magnezone has an ability called *Levitate* which makes it immune to Ground type attacks. Thus, having Magnezone only weak to **Fire** and **Fighting**.

With these information at hand, we can conclude that this team's primary weakness are **Fighting**, **Flying**, and **Poison**.

Now that we answered the first half of the question, let's now take a look at each Pokémon's **base stats**. (Thanks to Moomin).

In [ ]:
# Code taken from: 
# https://www.kaggle.com/wenxuanchen/d/abcsds/pokemon/pokemon-visualization-radar-chart-t-sne

TYPE_LIST = ['Grass','Fire','Water','Bug','Normal','Poison',
            'Electric','Ground','Fairy','Fighting','Psychic',
            'Rock','Ghost','Ice','Dragon','Dark','Steel','Flying']

COLOR_LIST = ['#8ED752', '#F95643', '#53AFFE', '#C3D221', '#BBBDAF', '#AD5CA2', 
              '#F8E64E', '#F0CA42', '#F9AEFE', '#A35449', '#FB61B4', '#CDBD72', 
              '#7673DA', '#66EBFF', '#8B76FF', '#8E6856', '#C3C1D7', '#75A4F9']

# The colors are copied from this script: 
# https://www.kaggle.com/ndrewgele/d/abcsds/pokemon/visualizing-pok-mon-stats-with-seaborn
# The colors look reasonable in this map: For example, Green for Grass, Red for Fire, Blue for Water...
COLOR_MAP = dict(zip(TYPE_LIST, COLOR_LIST))


# A radar chart example: 
# http://datascience.stackexchange.com/questions/6084/how-do-i-create-a-complex-radar-chart
def _scale_data(data, ranges):
    (x1, x2), d = ranges[0], data[0]
    return [(d - y1) / (y2 - y1) * (x2 - x1) + x1 for d, (y1, y2) in zip(data, ranges)]

class RaderChart():
    def __init__(self, fig, variables, ranges, n_ordinate_levels = 6):
        angles = np.arange(0, 360, 360./len(variables))

        axes = [fig.add_axes([0.1,0.1,0.8,0.8],polar = True, label = "axes{}".format(i)) for i in range(len(variables))]
        _, text = axes[0].set_thetagrids(angles, labels = variables)
        
        for txt, angle in zip(text, angles):
            txt.set_rotation(angle - 90)
        
        for ax in axes[1:]:
            ax.patch.set_visible(False)
            ax.xaxis.set_visible(False)
            ax.grid("off")
        
        for i, ax in enumerate(axes):
            grid = np.linspace(*ranges[i], num = n_ordinate_levels)
            grid_label = [""]+[str(int(x)) for x in grid[1:]]
            ax.set_rgrids(grid, labels = grid_label, angle = angles[i])
            ax.set_ylim(*ranges[i])
        
        self.angle = np.deg2rad(np.r_[angles, angles[0]])
        self.ranges = ranges
        self.ax = axes[0]

    def plot(self, data, *args, **kw):
        sdata = _scale_data(data, self.ranges)
        self.ax.plot(self.angle, np.r_[sdata, sdata[0]], *args, **kw)

    def fill(self, data, *args, **kw):
        sdata = _scale_data(data, self.ranges)
        self.ax.fill(self.angle, np.r_[sdata, sdata[0]], *args, **kw)

    def legend(self, *args, **kw):
        self.ax.legend(*args, **kw)

# select display colors according to Pokemon's Type 1
def select_color(types):
    colors = [None] * len(types)
    used_colors = set()
    for i, t in enumerate(types):
        curr = COLOR_MAP[t]
        if curr not in used_colors:
            colors[i] = curr
            used_colors.add(curr)
    unused_colors = set(COLOR_LIST) - used_colors
    for i, c in enumerate(colors):
        if not c:
            try:
                colors[i] = unused_colors.pop()
            except:
                raise Exception('Attempt to visualize too many pokemons. No more colors available.')
    return colors

df = pd.read_csv('../input/Pokemon.csv')

# In this order, 
# HP, Defense and Sp. Def will show on left; They represent defense abilities
# Speed, Attack and Sp. Atk will show on right; They represent attack abilities
# Attack and Defense, Sp. Atk and Sp. Def will show on opposite positions
use_attributes = ['Speed', 'Sp. Atk', 'Defense', 'HP', 'Sp. Def', 'Attack']

# choose the pokemons you like
use_pokemons = ['Azumarill', 'Torterra', 'Snorlax', 'Emboar', 'Magnezone']

df_plot = df[df['Name'].map(lambda x:x in use_pokemons)==True]
datas = df_plot[use_attributes].values 
ranges = [[2**-20, df_plot[attr].max()] for attr in use_attributes]
colors = select_color(df_plot['Type 1']) # select colors based on pokemon Type 1 

fig = plt.figure(figsize=(10, 10))
radar = RaderChart(fig, use_attributes, ranges)
for data, color, pokemon in zip(datas, colors, use_pokemons):
    radar.plot(data, color = color, label = pokemon)
    radar.fill(data, alpha = 0.1, color = color)
    radar.legend(loc = 1, fontsize = 'small')
plt.title('Base Stats of '+(', '.join(use_pokemons[:-1])+' and '+use_pokemons[-1] if len(use_pokemons)>1 else use_pokemons[0]))
plt.show()

With this data, we can see their total per stat. Magnezone having the highest base Attack and Speed, followed by Emboar with Sp. Attack and Defense, and Azumarill with the highest HP and Sp. Def.

All-in-all, their total stat combined is** 2,548** 

Dividing that by 5 (Total number of Pokémon in party) will give us **509.6**

Knowing these can answer the other half of the question, which are the top 5 Pokémon that can fill up the slot?

Let's first scout the Pokémon with a power level of **510 - 520**, since those Pokémon would fill in the roster very well. (Also, let's avoid Legendaries!)

In [ ]:
# This one sorts the table where in the lowest Total will be 510, and the highest Total will be 520.
pokeTablesorted = pokeTable[(pokeTable['Total'] >= 510) & (pokeTable['Total'] <= 520)]

pokeTablesorted.head(35)

As you can see, there are about **34** or more Pokémon available for scouting from Generations 1 - 6.

However, we need to take note of the team's weaknesses which are **Fighting**, **Flying**, and **Poison**. 

That means, we need to select the Top 5 Pokémon that are **STRONG** against the types we are weak to. Also, we need a Pokémon with decent **SPEED** stats since our team right now is very slow. 

Basing on the table above, let's select the Top 5 Pokémon that fits the standards of the team:

1.  Starmie (Water/Psychic)
2. Gardevoir (Psychic/Fairy)
3. Frost Rotom (Electric/Ice)
4. Klinklang (Steel)
5. Avalugg (Ice)

Let's use the graph above (the one we used on the 5 Pokemon on our roster) to compare these 5 Pokémon's stats and see who can we actually pick amongst the five.

In [ ]:
# Code taken from: 
# https://www.kaggle.com/wenxuanchen/d/abcsds/pokemon/pokemon-visualization-radar-chart-t-sne

TYPE_LIST = ['Grass','Fire','Water','Bug','Normal','Poison',
            'Electric','Ground','Fairy','Fighting','Psychic',
            'Rock','Ghost','Ice','Dragon','Dark','Steel','Flying']

COLOR_LIST = ['#8ED752', '#F95643', '#53AFFE', '#C3D221', '#BBBDAF', '#AD5CA2', 
              '#F8E64E', '#F0CA42', '#F9AEFE', '#A35449', '#FB61B4', '#CDBD72', 
              '#7673DA', '#66EBFF', '#8B76FF', '#8E6856', '#C3C1D7', '#75A4F9']

# The colors are copied from this script: 
# https://www.kaggle.com/ndrewgele/d/abcsds/pokemon/visualizing-pok-mon-stats-with-seaborn
# The colors look reasonable in this map: For example, Green for Grass, Red for Fire, Blue for Water...
COLOR_MAP = dict(zip(TYPE_LIST, COLOR_LIST))


# A radar chart example: 
# http://datascience.stackexchange.com/questions/6084/how-do-i-create-a-complex-radar-chart
def _scale_data(data, ranges):
    (x1, x2), d = ranges[0], data[0]
    return [(d - y1) / (y2 - y1) * (x2 - x1) + x1 for d, (y1, y2) in zip(data, ranges)]

class RaderChart():
    def __init__(self, fig, variables, ranges, n_ordinate_levels = 6):
        angles = np.arange(0, 360, 360./len(variables))

        axes = [fig.add_axes([0.1,0.1,0.8,0.8],polar = True, label = "axes{}".format(i)) for i in range(len(variables))]
        _, text = axes[0].set_thetagrids(angles, labels = variables)
        
        for txt, angle in zip(text, angles):
            txt.set_rotation(angle - 90)
        
        for ax in axes[1:]:
            ax.patch.set_visible(False)
            ax.xaxis.set_visible(False)
            ax.grid("off")
        
        for i, ax in enumerate(axes):
            grid = np.linspace(*ranges[i], num = n_ordinate_levels)
            grid_label = [""]+[str(int(x)) for x in grid[1:]]
            ax.set_rgrids(grid, labels = grid_label, angle = angles[i])
            ax.set_ylim(*ranges[i])
        
        self.angle = np.deg2rad(np.r_[angles, angles[0]])
        self.ranges = ranges
        self.ax = axes[0]

    def plot(self, data, *args, **kw):
        sdata = _scale_data(data, self.ranges)
        self.ax.plot(self.angle, np.r_[sdata, sdata[0]], *args, **kw)

    def fill(self, data, *args, **kw):
        sdata = _scale_data(data, self.ranges)
        self.ax.fill(self.angle, np.r_[sdata, sdata[0]], *args, **kw)

    def legend(self, *args, **kw):
        self.ax.legend(*args, **kw)

# select display colors according to Pokemon's Type 1
def select_color(types):
    colors = [None] * len(types)
    used_colors = set()
    for i, t in enumerate(types):
        curr = COLOR_MAP[t]
        if curr not in used_colors:
            colors[i] = curr
            used_colors.add(curr)
    unused_colors = set(COLOR_LIST) - used_colors
    for i, c in enumerate(colors):
        if not c:
            try:
                colors[i] = unused_colors.pop()
            except:
                raise Exception('Attempt to visualize too many pokemons. No more colors available.')
    return colors

df = pd.read_csv('../input/Pokemon.csv')

# In this order, 
# HP, Defense and Sp. Def will show on left; They represent defense abilities
# Speed, Attack and Sp. Atk will show on right; They represent attack abilities
# Attack and Defense, Sp. Atk and Sp. Def will show on opposite positions
use_attributes = ['Speed', 'Sp. Atk', 'Defense', 'HP', 'Sp. Def', 'Attack']

# choose the pokemons you like
use_pokemons = ['Starmie', 'Gardevoir', 'RotomFrost Rotom', 'Klinklang', 'Avalugg']

df_plot = df[df['Name'].map(lambda x:x in use_pokemons)==True]
datas = df_plot[use_attributes].values 
ranges = [[2**-20, df_plot[attr].max()] for attr in use_attributes]
colors = select_color(df_plot['Type 1']) # select colors based on pokemon Type 1 

fig = plt.figure(figsize=(10, 10))
radar = RaderChart(fig, use_attributes, ranges)
for data, color, pokemon in zip(datas, colors, use_pokemons):
    radar.plot(data, color = color, label = pokemon)
    radar.fill(data, alpha = 0.1, color = color)
    radar.legend(loc = 1, fontsize = 'small')
plt.title('Base Stats of '+(', '.join(use_pokemons[:-1])+' and '+use_pokemons[-1] if len(use_pokemons)>1 else use_pokemons[0]))
plt.show()

With this stat above, we can compare and analyze the results.

Avalugg has the **highest Attack**, **HP**, **and Defensive stat**, Avalugg is very slow compared to the other Pokémon. Even then, Avalugg's main type is Ice, which is only good with **Flying** type Pokémon. But he is super vulnerable with **Fighting,** so he is out of the picture.

Klinklang has an **all-around stat line** that finished 2nd in Speed. However, Klinklang is a Steel type Pokémon, which is good against **Poison** types since it doesn't affect Klinklang. Other than that, he's practically useless against **Fighting**.

Frost Rotom has another **decent all-around stat line**, finished 3rd in Speed and Defense, 2nd in Sp. Def and Sp. Atk, and 4th in HP. Frost Rotom is super effective when battling** Flying** type Pokémon, but then Frost Rotom is weak to **Fighting**.

Gardevoir is a Pokémon with the **highest Sp. Def and Sp. Atk** stat, a powerful addition to the team. However, Gardevoir is also one of the slowest. Gardevoir is very effecting against **Fighting** and **Poison** (with its Psychic moves) but very vulnerable to **Flying**.

Lastly, Starmie. The Pokémon with the **highest Speed** stat and its other stats are balanced. Starmie is the perfect Pokémon to add to the team because of it's **Water** and **Psychic** attributes/type that is strong against **Fighting**, **Poison**, and even **Flying** (Note: Starmie can learn Ice type moves).

To summarize, the 6th and final slot to fill in the team is **Starmie**. 

Starmie is **super effective** to the types the team is generally weak against, Starmie is also the Pokémon we need because the team lacks in Speed. Starmie provides us with that **Speed**, along with its other balanced stats of **Attack**, **Sp. Attack**, and **Sp. Defense**.

**Question 2**. Now that the last slot of my Pokémon team is filled, the team is complete.

However, I feel like there are some things I need to learn more about  what this team is capable of. For starters, who do I run first when I am matched up with a team filled with certain types that I am strong against, weak against, and there are teams where in they outspeed my team, or have a high defensive stat.

How should I tackle the rotations of my team in going up against these certain types?

Let's first look at the **stats of every Pokémon** available. (Same graph as earlier but with Starmie)

In [ ]:
# Code taken from: 
# https://www.kaggle.com/wenxuanchen/d/abcsds/pokemon/pokemon-visualization-radar-chart-t-sne

TYPE_LIST = ['Grass','Fire','Water','Bug','Normal','Poison',
            'Electric','Ground','Fairy','Fighting','Psychic',
            'Rock','Ghost','Ice','Dragon','Dark','Steel','Flying']

COLOR_LIST = ['#8ED752', '#F95643', '#53AFFE', '#C3D221', '#BBBDAF', '#AD5CA2', 
              '#F8E64E', '#F0CA42', '#F9AEFE', '#A35449', '#FB61B4', '#CDBD72', 
              '#7673DA', '#66EBFF', '#8B76FF', '#8E6856', '#C3C1D7', '#75A4F9']

# The colors are copied from this script: 
# https://www.kaggle.com/ndrewgele/d/abcsds/pokemon/visualizing-pok-mon-stats-with-seaborn
# The colors look reasonable in this map: For example, Green for Grass, Red for Fire, Blue for Water...
COLOR_MAP = dict(zip(TYPE_LIST, COLOR_LIST))


# A radar chart example: 
# http://datascience.stackexchange.com/questions/6084/how-do-i-create-a-complex-radar-chart
def _scale_data(data, ranges):
    (x1, x2), d = ranges[0], data[0]
    return [(d - y1) / (y2 - y1) * (x2 - x1) + x1 for d, (y1, y2) in zip(data, ranges)]

class RaderChart():
    def __init__(self, fig, variables, ranges, n_ordinate_levels = 6):
        angles = np.arange(0, 360, 360./len(variables))

        axes = [fig.add_axes([0.1,0.1,0.8,0.8],polar = True, label = "axes{}".format(i)) for i in range(len(variables))]
        _, text = axes[0].set_thetagrids(angles, labels = variables)
        
        for txt, angle in zip(text, angles):
            txt.set_rotation(angle - 90)
        
        for ax in axes[1:]:
            ax.patch.set_visible(False)
            ax.xaxis.set_visible(False)
            ax.grid("off")
        
        for i, ax in enumerate(axes):
            grid = np.linspace(*ranges[i], num = n_ordinate_levels)
            grid_label = [""]+[str(int(x)) for x in grid[1:]]
            ax.set_rgrids(grid, labels = grid_label, angle = angles[i])
            ax.set_ylim(*ranges[i])
        
        self.angle = np.deg2rad(np.r_[angles, angles[0]])
        self.ranges = ranges
        self.ax = axes[0]

    def plot(self, data, *args, **kw):
        sdata = _scale_data(data, self.ranges)
        self.ax.plot(self.angle, np.r_[sdata, sdata[0]], *args, **kw)

    def fill(self, data, *args, **kw):
        sdata = _scale_data(data, self.ranges)
        self.ax.fill(self.angle, np.r_[sdata, sdata[0]], *args, **kw)

    def legend(self, *args, **kw):
        self.ax.legend(*args, **kw)

# select display colors according to Pokemon's Type 1
def select_color(types):
    colors = [None] * len(types)
    used_colors = set()
    for i, t in enumerate(types):
        curr = COLOR_MAP[t]
        if curr not in used_colors:
            colors[i] = curr
            used_colors.add(curr)
    unused_colors = set(COLOR_LIST) - used_colors
    for i, c in enumerate(colors):
        if not c:
            try:
                colors[i] = unused_colors.pop()
            except:
                raise Exception('Attempt to visualize too many pokemons. No more colors available.')
    return colors

df = pd.read_csv('../input/Pokemon.csv')

# In this order, 
# HP, Defense and Sp. Def will show on left; They represent defense abilities
# Speed, Attack and Sp. Atk will show on right; They represent attack abilities
# Attack and Defense, Sp. Atk and Sp. Def will show on opposite positions
use_attributes = ['Speed', 'Sp. Atk', 'Defense', 'HP', 'Sp. Def', 'Attack']

# choose the pokemons you like
use_pokemons = ['Groudon', 'Vileplume', 'Typhlosion', 'Weavile', 'Rayquaza', 'Starmie']

df_plot = df[df['Name'].map(lambda x:x in use_pokemons)==True]
datas = df_plot[use_attributes].values 
ranges = [[2**-20, df_plot[attr].max()] for attr in use_attributes]
colors = select_color(df_plot['Type 1']) # select colors based on pokemon Type 1 

fig = plt.figure(figsize=(10, 10))
radar = RaderChart(fig, use_attributes, ranges)
for data, color, pokemon in zip(datas, colors, use_pokemons):
    radar.plot(data, color = color, label = pokemon)
    radar.fill(data, alpha = 0.1, color = color)
    radar.legend(loc = 1, fontsize = 'small')
plt.title('Base Stats of '+(', '.join(use_pokemons[:-1])+' and '+use_pokemons[-1] if len(use_pokemons)>1 else use_pokemons[0]))
plt.show()

Adding Starmie to the team added us with a higher **Attack** stat (now that Starmie is in the lead with 123), and is the second to **Speed**, **Sp. Atk**, and total **HP**. With this addition, we have a more stable line-up because **4 out of 6 Pokémon** leads in a certain category (namely Starmie, Torterra, Azumarill, and Magnezone). 

With the knowledge of the **Type weakness** and the **overall** stat of the team, we can find a rotation that fits best in particular situations.

Let's take a look at the overall average of every Pokémon type and their stats. **(With the help of Andrew Gelé's data)**

In [ ]:
# Code from: https://www.kaggle.com/ndrewgele/visualizing-pok-mon-stats-with-seaborn

# His method of coding will make the seaborn plot work, seaborn plot will let us see
# The total pokemon number and in which stat are they in.
pkmn = pd.read_csv('../input/Pokemon.csv')

pkmn = pkmn.drop(['Total', '#', 'Generation', 'Legendary'], 1)

pkmn.head(10)

In [ ]:
# Code from: https://www.kaggle.com/ndrewgele/visualizing-pok-mon-stats-with-seaborn

pkmn = pd.melt(pkmn, id_vars=["Name", "Type 1", "Type 2"], var_name="Stat")

pkmn.head(10)

In [ ]:
# Code from: https://www.kaggle.com/ndrewgele/visualizing-pok-mon-stats-with-seaborn

sns.set_style("whitegrid")
with sns.color_palette([
    "#8ED752", "#F95643", "#53AFFE", "#C3D221", "#BBBDAF",
    "#AD5CA2", "#F8E64E", "#F0CA42", "#F9AEFE", "#A35449",
    "#FB61B4", "#CDBD72", "#7673DA", "#66EBFF", "#8B76FF",
    "#8E6856", "#C3C1D7", "#75A4F9"], n_colors=18, desat=.9):
    plt.figure(figsize=(12,10))
    plt.ylim(0, 275)
    sns.swarmplot(x="Stat", y="value", data=pkmn, hue="Type 1", dodge=True, size=7)
    plt.legend(bbox_to_anchor=(1, 1), loc=2, borderaxespad=0.);

Thanks to Andrew Gelé, we now have a seaborn plot of all the possible Pokémon that our team will face (including their own selves) and their stats on HP, Attack, Defense, Sp. Atk, Sp. Def, and Speed!

With the knowledge of having all **Pokémon's base stat** in a seaborn plot, and the **type weaknesses** of every type, we can adapt our lineup based on the Pokémon's **type, stat, strengths, and weaknesses**.

Question 3: I'm building a competitive team for Pokémon, but I want to put one with a great evolution. I don't want to have a Pokémon who grows in Attack but still remains weak when it comes to Defense. So, with the evolution lines in mind, who is the most stable when it comes to stat growth?

***EXPLAIN THOUGHT PROCESS ONLY, NO NEED FOR GRAPHS. ***

First things first, we need to add a **new column** in the existing table that tells which **stage of evolution the Pokémon is**. For example, Charmander is Stage 1, Charmeleon is Stage 2, and Charizard is Stage 3.

Next, we **arrange the table** according to their Stage in **ascending** order so that we would be able to identify all the Pokémon that are Stage 1, Stage 2, and Stage 3.

Then, we make a **line graph** per Stage to determine their stats based on their HP, Attack, Defense, Sp. Atk, Sp. Def, and Speed. So that we would see the behavior of each Pokémon as they evolve.

After doing so, we find the **top 5 Pokémon** that fits the standards of the question; which is to find a Pokémon that has a balanced growth and not a Pokémon that is only growing in one stat.